In [21]:
import mysql.connector
from mysql.connector import errorcode
import pandas as pd
import numpy as np

In [22]:
# 🏆 "OSCARS" TABLE --> DEFINE ✍️ FUNCTIONS

# Function to insert 'list_oscars_BS' into the 'oscars' table.
def insert_oscars():
    # Connection
    cnx = mysql.connector.connect(user='root', password='AlumnaAdalab', host='127.0.0.1', database='cinem_extract')

    # Cursor creation
    mycursor = cnx.cursor()

    try: 
        # Read CSV file
        df = pd.read_csv('data/oscars/list_oscars_BS.csv') 

        # SQL to insert data. You must indicate the name of the columns in the first parenthesis and in values enter the number of columns with %s
        sql_insert_query = """ INSERT INTO oscars (ceremony_year, best_movie, best_director, best_actor, best_actress) VALUES (%s, %s, %s, %s, %s)"""

        # Iterate over each row of the DataFrame and insert the data for each column.
        for _, row in df.iterrows():
            mycursor.execute(sql_insert_query, (row['ceremony_year'], row['best_movie'], row['best_director'], row['best_actor'], row['best_actress'])) 

        # Confirm changes with commit
        cnx.commit() 
        print(f"{mycursor.rowcount} registro(s) insertado(s)📝") 

    except mysql.connector.Error as err: 
        print(f"Error al insertar en la base de datos: {err} ❌") 

    finally: 
        if cnx.is_connected(): 
            mycursor.close() 
            cnx.close() 
            print("Conexión a la base de datos cerrada🔚")

In [23]:
# 🏆 "OSCARS" TABLE --> EXECUTE ▶️ FUNCTIONS

# call the insert function for the oscars table
insert_oscars()

1 registro(s) insertado(s)📝
Conexión a la base de datos cerrada🔚


In [24]:
# 🎭 "ACTOR DETAILS" TABLE --> DEFINE ✍️ FUNCTIONS

# function for cleaning actor_details .csv: replaces 'none' and NaN with 0

def clean_ad_data (genre, start_page, end_page):
    #1- read csv
    df = pd.read_csv(f'data/selenium_actors/list_{genre}_range_{start_page}_{end_page}_sel_actor.csv')
    print (f"data/selenium_actors/list_{genre}_range_{start_page}_{end_page}_sel_actor.csv opened")
    
    #2- find and replace 'none' with None
    df["birth_year"].replace("none", 0, inplace=True)
    df.fillna(value=0, inplace=True)
    df.set_index('actor_name', inplace=True)
    print ("csv updated")
    
    #3 save the updated data as .csv
    df.to_csv(f'data/selenium_actors/clean/list_{genre}_range_{start_page}_{end_page}_sel_actor.csv')
    print (f"data/selenium_actors/clean/list_{genre}_range_{start_page}_{end_page}_sel_actor.csv CREATED✅")

# function to insert the data into 'actor_details' table

def insert_actor_details(genre, start_page, end_page):
    cnx = mysql.connector.connect(user='root', password='AlumnaAdalab', host='127.0.0.1', database='cinem_extract')

    mycursor = cnx.cursor()

    try: 
        df = pd.read_csv(f'data/selenium_actors/clean/list_{genre}_range_{start_page}_{end_page}_sel_actor.csv')

        sql_insert_query = """ 
        INSERT INTO actor_details (actor_name, birth_year, known_for, actor_role, actor_awards)
        VALUES (%s, %s, %s, %s, %s) 
        ON DUPLICATE KEY UPDATE 
            actor_name = VALUES(actor_name),
            birth_year = VALUES(birth_year),
            known_for = VALUES(known_for),
            actor_role = VALUES(actor_role),
            actor_awards = VALUES(actor_awards)""" 

        for _, row in df.iterrows():
            mycursor.execute(sql_insert_query, (row['actor_name'],row['birth_year'],row['known_for'],row['actor_role'],row['actor_awards'])) 

        cnx.commit() 
        print(f"{mycursor.rowcount} registro(s) insertado(s)📝") 

    except mysql.connector.Error as err: 
        print(f"Error al insertar en la base de datos: {err} ❌") 

    finally: 
        if cnx.is_connected(): 
            mycursor.close() 
            cnx.close() 
            print("Conexión a la base de datos cerrada🔚")

In [ ]:
# 🎭 "ACTOR DETAILS" TABLE --> DEFINE ✍️ FUNCTIONS (THIS IS TO ADD ACTORS NAMES ONLY)

def insert_actor_name_only():
    cnx = mysql.connector.connect(user='root', password='AlumnaAdalab', host='127.0.0.1', database='cinem_extract')

    mycursor = cnx.cursor()

    try: 
        df = pd.read_csv(f'data/oscars/actor_details_oscars_only.csv')

        sql_insert_query = """ 
        INSERT INTO actor_details (actor_name)
        VALUES (%s)
        ON DUPLICATE KEY UPDATE 
            actor_name = VALUES(actor_name)""" 

        for _, row in df.iterrows():
            mycursor.execute(sql_insert_query, (row['actor_name'],)) 

        cnx.commit() 
        print(f"{mycursor.rowcount} registro(s) insertado(s)📝") 

    except mysql.connector.Error as err: 
        print(f"Error al insertar en la base de datos: {err} ❌") 

    finally: 
        if cnx.is_connected(): 
            mycursor.close() 
            cnx.close() 
            print("Conexión a la base de datos cerrada🔚")

In [25]:
#execute function

insert_actor_name_only()

1 registro(s) insertado(s)📝
Conexión a la base de datos cerrada🔚


In [41]:
# 🎭 "ACTOR DETAILS" TABLE --> EXECUTE ▶️ FUNCTIONS


# enter the correspondent genre and start and end page
genre = 'oscars'       # enter "action" "comedy" "drama" "oscars"
start_page = 1
end_page = 2

# iterate over the list of files
for i in range(start_page, end_page):
    
    print(f"Iteracion {i}")
    
    # clean data
    clean_ad_data (genre, i, i+1)

    # insert data
    insert_actor_details(genre, i, i+1)
    

Iteracion 1
data/selenium_actors/list_oscars_range_1_2_sel_actor.csv opened
csv updated
data/selenium_actors/clean/list_oscars_range_1_2_sel_actor.csv CREATED✅
1 registro(s) insertado(s)📝
Conexión a la base de datos cerrada🔚


In [27]:
#  🎭🏆 "ACTOR AWARDS" TABLE --> DEFINE ✍️ FUNCTIONS

# Function to insert the data from 'actor_awards.csv' into the 'actor_awards' table. 
# It will not work if we don't have data in the 'actor_details' table.
def insert_actor_awards():
    cnx = mysql.connector.connect(user='root', password='AlumnaAdalab', host='127.0.0.1', database='cinem_extract')

    mycursor = cnx.cursor()

    try: 
        df = pd.read_csv('data/oscars/actor_awards.csv')
    
        sql_insert_query = """ INSERT INTO actor_awards (actor_name, ceremony_year, award_type) VALUES (%s, %s, %s)"""

        for _, row in df.iterrows():
            mycursor.execute(sql_insert_query, (row['actor_name'], row['ceremony_year'], row['award_type'])) 

        cnx.commit() 
        print(f"{mycursor.rowcount} registro(s) insertado(s)📝") 

    except mysql.connector.Error as err: 
        print(f"Error al insertar en la base de datos: {err} ❌") 

    finally: 
        if cnx.is_connected(): 
            mycursor.close() 
            cnx.close() 
            print("Conexión a la base de datos cerrada🔚")

In [28]:
#  🎭🏆 "ACTOR AWARDS" TABLE --> EXECUTE ▶️ FUNCTIONS

# call the insert function for the oscars table
insert_actor_awards()

In [29]:
# 🎬🏆 "RELATION MOVIE OSCARS" TABLE --> DEFINE ✍️ FUNCTIONS

# insert data into relation_movie_oscars

def insert_relation_movie_oscars():
    # Connection
    cnx = mysql.connector.connect(user='root', password='AlumnaAdalab', host='127.0.0.1', database='cinem_extract')

    # Cursor creation
    mycursor = cnx.cursor()

    try: 
        # Read CSV file
        df = pd.read_csv('data/oscars/relation_movie_oscars.csv')

        # SQL to insert data. You must indicate the name of the columns in the first parenthesis and in values enter the number of columns with %s
        sql_insert_query = """ INSERT INTO relation_movie_oscars (id_imdb, best_movie, ceremony_year) VALUES (%s, %s, %s)"""

        # Iterate over each row of the DataFrame and insert the data for each column.
        for _, row in df.iterrows():
            mycursor.execute(sql_insert_query, (row['id_imdb'], row['best_movie'], row['ceremony_year'])) 

        # Confirm changes with commit
        cnx.commit() 
        print(f"{mycursor.rowcount} registro(s) insertado(s)📝")

    except mysql.connector.Error as err:
        print(f"Error al insertar en la base de datos: {err} ❌") 

    finally: 
        if cnx.is_connected(): 
            mycursor.close() 
            cnx.close() 
            print("Conexión a la base de datos cerrada🔚")


In [42]:
# 🎬🏆 "RELATION MOVIE OSCARS" TABLE --> EXECUTE ▶️ FUNCTIONS

# call the insert function for relation_movie_oscars

insert_relation_movie_oscars()

In [31]:
# 📽️ "MOVIE DETAILS" TABLE --> DEFINE ✍️ FUNCTIONS

# function to clean data

def clean_md_data (genre, start_page, end_page):
    #1- read csv
    df = pd.read_csv(f'data/selenium_movies/list_{genre}_{start_page}_{end_page}_sel_movies copy.csv') #remove "copy" it's only for testing
    print (f"\ndata/selenium_movies/list_{genre}_{start_page}_{end_page}_sel_movies.csv opened")
    
    #2- find and replace 'none' with None
    df["score_imdb"].replace("none", 0, inplace=True)
    df["score_imdb"] = df["score_imdb"].str.replace(',', '.').astype(float)
    df["score_rt"].replace("none", 0, inplace=True)
    df["duration_imdb"].replace("none", 0, inplace=True)
    df.fillna(value=0, inplace=True)
    df.set_index('id_imdb', inplace=True)
    print ("\ncsv updated")
    
    #3 save the updated data as .csv
    df.to_csv(f'data/selenium_movies/clean/list_{genre}_{start_page}_{end_page}_sel_movies.csv')
    print (f"\ndata/selenium_movies/clean/list_{genre}_{start_page}_{end_page}_sel_movies.csv CREATED✅")

# function to insert the data into 'movie_details' table

def insert_movie_details(genre, start_page, end_page):
    cnx = mysql.connector.connect(user='root', password='AlumnaAdalab', host='127.0.0.1', database='cinem_extract')

    mycursor = cnx.cursor()

    try:
        df = pd.read_csv(f'data/selenium_movies/clean/list_{genre}_{start_page}_{end_page}_sel_movies.csv')

        sql_insert_query = """ 
        INSERT INTO movie_details (id_imdb, score_imdb, score_rt, director_imdb, screenwriters_imdb, plot_rt, duration_imdb, title_imdb)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE 
            id_imdb = VALUES(id_imdb),
            score_imdb = VALUES(score_imdb),
            score_rt = VALUES(score_rt),
            director_imdb = VALUES(director_imdb),
            screenwriters_imdb = VALUES(screenwriters_imdb),
            plot_rt = VALUES(plot_rt),
            duration_imdb = VALUES(duration_imdb),
            title_imdb = VALUES (title_imdb)"""

        for _, row in df.iterrows():
            mycursor.execute(sql_insert_query, (row['id_imdb'],row['score_imdb'],row['score_rt'],row['director_imdb'],row['screenwriters_imdb'],row['plot_rt'],row['duration_imdb'],row['title_imdb'])) 

        cnx.commit() 
        print(f"{mycursor.rowcount} registro(s) insertado(s)📝") 

    except mysql.connector.Error as err: 
        print(f"Error al insertar en la base de datos: {err} ❌") 

    finally: 
        if cnx.is_connected(): 
            mycursor.close() 
            cnx.close() 
            print("Conexión a la base de datos cerrada🔚")

In [43]:
# 📽️ "MOVIE DETAILS" TABLE --> EXECUTE ▶️ FUNCTIONS

# IN PROGRESS Having issues here because not all the imdb_id are uploaded 

# enter the correspondent genre and start and end page
genre = 'action'       # enter "action" "comedy" "drama" "oscars"
start_page = 1
end_page = 2

# iterate over the list of files
for i in range(start_page, end_page):
    
    print(f"Iteracion {i}")
    # clean data 
    clean_md_data(genre, i, i+1)
    # insert data
    insert_movie_details(genre, i, i+1)

Iteracion 1

data/selenium_movies/list_action_1_2_sel_movies.csv opened

csv updated

data/selenium_movies/clean/list_action_1_2_sel_movies.csv CREATED✅
Error al insertar en la base de datos: 1265 (01000): Data truncated for column 'score_imdb' at row 1 ❌
Conexión a la base de datos cerrada🔚


In [32]:
# 🎬 "MOVIES" TABLE --> DEFINE ✍️ FUNCTIONS

# insert data into movies

def clean_and_prepare_data(file_path):
    try:
        df = pd.read_csv(file_path)
        
        # Establecer 'id_imdb' como índice del DataFrame
        df.set_index('id_imdb', inplace=True)

        # Reemplazar NaN con valores por defecto
        df['release_month'].fillna(0, inplace=True)  # O algún otro valor por defecto apropiado
        df['type'].fillna('not', inplace=True)  # O algún otro valor por defecto apropiado
        df['genre'].fillna('not', inplace=True)  # O algún otro valor por defecto apropiado

        # Convertir 'none' a None para que se traduzcan a NULL en SQL
        df.replace('none', None, inplace=True)

        # Convertir NaN a None para que se traduzcan a NULL en SQL
        df = df.where(pd.notnull(df), None)

        # Verificar los tipos de datos y reemplazar valores no válidos
        df['release_month'] = df['release_month'].astype(int, errors='ignore')
        df['release_year'] = df['release_year'].astype(int, errors='ignore')

        # Imprimir los primeros registros para depuración
        print(df.head())

        # Contar las ocurrencias de 'not' en las columnas 'type' y 'genre'
        type_not_count = df['type'].value_counts().get('not', 0)
        genre_not_count = df['genre'].value_counts().get('not', 0)

        print(f"Ocurrencias de 'not' en 'type': {type_not_count}")
        print(f"Ocurrencias de 'not' en 'genre': {genre_not_count}")

        return df

    except FileNotFoundError as fnf_error:
        print(f"Error al buscar archivo: {fnf_error}")
        return None


def insert_movies(df):
    cnx = mysql.connector.connect(user='root', password='AlumnaAdalab', host='127.0.0.1', database='cinem_extract')
    mycursor = cnx.cursor()

    try:
        sql_insert_query = """ 
        INSERT INTO movies (id_imdb, type, name, release_year, release_month, genre) 
        VALUES (%s, %s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE 
            id_imdb = VALUES(id_imdb),
            type = VALUES(type),
            name = VALUES(name),
            release_year = VALUES(release_year),
            release_month = VALUES(release_month),
            genre = VALUES(genre)"""

        rowcount_before = mycursor.rowcount

        for id_imdb, row in df.iterrows():
            mycursor.execute(sql_insert_query, (id_imdb, row['type'], row['name'], row['release_year'], row['release_month'], row['genre'])) 

        cnx.commit()

        rowcount_after = mycursor.rowcount
        inserted_count = rowcount_after - rowcount_before

        print(f"{inserted_count} registro(s) insertado(s)📝")

    except mysql.connector.Error as err:
        print(f"Error al insertar en la base de datos: {err}")

    finally:
        if cnx.is_connected():
            mycursor.close()
            cnx.close()
            print("Conexión a la base de datos cerrada🔚")

In [38]:
# 🎬 "MOVIES" TABLE --> EXECUTE ▶️ FUNCTIONS

# call the insert function for movies

genre = 'oscars'
start_page = 1
end_page = 2

for i in range(start_page, end_page):
    print(f"Iteracion {i}")
    file_path = f'data/api/list_{genre}_range_{i}_{i+1}_API.csv'
    
    df = clean_and_prepare_data(file_path)
    
    if df is None or df.empty:
        print(f"No se encontraron datos válidos en {file_path}")
        continue
    
    insert_movies(df)

Iteracion 1
            type                                           name  release_year  \
id_imdb                                                                         
tt0167260  Movie  The Lord of the Rings: The Return of the King          2003   
tt0169547  Movie                                American Beauty          1999   
tt0172495  Movie                                      Gladiator          2000   
tt0268978  Movie                               A Beautiful Mind          2001   
tt0299658  Movie                                        Chicago          2002   

           release_month    genre  
id_imdb                            
tt0167260             12  Fantasy  
tt0169547              1   Comedy  
tt0172495              5    Drama  
tt0268978              2    Drama  
tt0299658              2  Musical  
Ocurrencias de 'not' en 'type': 0
Ocurrencias de 'not' en 'genre': 0
2 registro(s) insertado(s)📝
Conexión a la base de datos cerrada🔚
